# Converting the csv files to a SQL database

In [1]:
import pandas as pd
import sqlite3
import os
import re

# my code
from addCSVToSQLdb import addCSVToSQLdb

In [2]:
dbName = 'CHiMaD_SDC.db'

In [8]:
path = '../csv/'
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
for csvfile in files:
    print(csvfile)
    addCSVToSQLdb(dbName, csvfile, path)

cjebcpsfordsa.csv
coalloywhitneyt.csv
cobaltbasedsuperalloysbrandon.csv
cobaltbasedsuperalloysbrandonfinished.csv
cobasedsuperalloysbrandon3.csv
cobasedsuperalloysbrandon4.csv
dsafinalmaybe.csv
emeryexam1m54.csv
metalamslm.csv
newparagraph.csv
paragraphs.csv
pecs.csv
pecsclass.csv
polymercompositeexample.csv
polymercompositeexample1.csv
superalloytest.csv
test.csv
thermoelectricsjm.csv
thermoelectricstest.csv
whitneydsa.csv
blank.csv
cjedsarandombcps.csv
aluminumchimadexample.csv
emeryexam1m541.csv


## Test selecting everything from the database and puting back into DataFrame

In [10]:
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]

for t in tables:
    print(t)
    cursor.execute('SELECT * FROM ' + t)
    columns = [description[0] for description in cursor.description]
    df2 = pd.DataFrame(cursor.fetchall(), columns = columns)    
    display(df2)


cjebcpsfordsa


,Timestamp,IP,username,version,task,microphaseseparate,identityofblocks,specificanddefinedpatterns,appliedtoasurface,thermallyannealed,...,segregationstrengthofthebcpn,chemicalidentityofmonomers,thermodynamic,specifieddimensions,kinetics,selfdiffusion,structuralfluctuations,structuraluniformity,structuralreorganization,annihilationofdefects


coalloywhitneyt


,Timestamp,IP,username,version,task,hightemperature,twophaseggmicrostructure,cuboidalprecipitates,creepresistance,highermeltingpoint,corrosionresistance,composition,elasticenergy,chemicalenergy,precipitatemorphology,additivemanufacturable


cobaltbasedsuperalloysbrandon


...

,Timestamp,IP,username,version,task,ingotmetallurgy,ingotrefining,deoxidation,ingotsolidification,feandotherimpuritiesformcoarseinclusionorconstituentphasessuchasal3fe,...,hotworking,solutiontreatment,quenched,aged,precipitatecugpzones,finalmatrix,elasticmodulus,yieldstrength,fracturetoughness,grainsize


emeryexam1m541


,Timestamp,IP,username,version,task,figureofmeritzt,latticethermalconductivity,electronicthermalconductivity,carriermobility,carrierconcentration,...,densityofstates,dopingthematrix,latticethermalconductivityresistance,microstructuregrainboundariessecondaryphases,castthealloyasaningot,consolidationhomogenizationstepiesparkplasmasinteringhotpressing,annealingtreatment,grainstructureincludinggbchemistry,nanostructureofthesecondaryphases,phononscattering


polymercompositeexample


,Timestamp,IP,username,version,task,lightweightmechanical,toughness,strengthstiffness,lowdensity,extremeenvironments,...,thechoiceandpreparationofreinforcement,morphology,microstructureofthereinforcementphase,layupandcuring,dispersion,aggregation,orientationofthereinforcement,postlayupannealingandaging,matrixreinforcementinterface,failuremode
0,11/1/2021 9:29:25,00.000.00.00,JonEmery,1,para,Property,Property,Property,Property,Structure,...,Property,Property,Property,Structure,Property,Structure,Property,Performance,Structure,Processing
1,11/1/2021 13:41:39,00.000.00.00,JonEmery,2,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
2,11/1/2021 9:33:38,00.000.00.00,JonEmery,1,SDC,None,resistancetofracture,lightweightmechanical,lightweightmechanical,None,...,microstructureofthereinforcementphase heteroge...,thermalexpansioncoefficientmismatch,glasstransitiontemperaturetg,aggregation microstructureofthereinforcementphase,slowdegradationrateswhenexposedtoo2h2oandsalts,thermalexpansioncoefficientmismatch,glasstransitiontemperaturetg,matrixreinforcementinterface aggregation,failuremode,resistancetofracture
3,11/1/2021 13:15:01,00.000.00.00,WhitneyT,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
4,11/1/2021 13:15:09,00.000.00.00,chris_h,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Performance
5,11/1/2021 13:15:12,00.000.00.00,AaronGeller,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
6,11/1/2021 13:16:08,00.000.00.00,Alex,1,para,Performance,Property,Property,Property,Performance,...,Processing,Property,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Structure
7,11/1/2021 13:16:09,00.000.00.00,JamesMale,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
8,11/1/2021 13:17:18,00.000.00.00,BrandonO,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
9,11/1/2021 13:17:35,00.000.00.00,Kuma,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Processing,Property


In [5]:
cursor.close()

# Manual testing

## Read in a csv file

In [ ]:
csvName = 'thermoelectricsjm.csv' # this has a column with ".1" in the name, and sqlite does not allow it
df = pd.read_csv('../' + csvName)
df

## Create a SQL database

In [ ]:
dbName = 'CHiMaD_SDC.db'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

In [ ]:
# the database name will be the file name w/o csv
# take only alphanumeric values
tableName = re.sub(r'\W+', '',csvName.replace('.csv','')) 
tableName

In [ ]:
# include all columns, and assume that they are all text
cols = ''
for col in df.columns:
    cc = re.sub(r'\W+', '', col) # only alphanumeric values in column names
    cols += cc + ' text, '
cols = cols[:-2]
cols

In [ ]:
cursor.execute('CREATE TABLE IF NOT EXISTS ' + tableName + ' (' + cols + ')')
conn.commit()

In [ ]:
# add the dataframe into the database
df.to_sql(tableName, conn, if_exists='replace', index = False)

In [ ]:
# command to copy files from Downloads and change names
#find ~/Downloads/ | grep testParagraph | awk '{print ("cp "$0" "$3)}' | sed s/" - "/"\\\ -\\\ "/ > foo